In [2]:
import os
os.chdir("/data/t-achetan/adv_geom/src/")
import torch
import torch.optim as optim

from expB.myNNs import MTMLPwithNormalisation

In [5]:
# fn = "/data/t-achetan_2/dumps/expC_dist_learner_for_adv_ex/rdm_concspheres_test/rdm_concspheres_k50n500_noninfdist_moreoffmfldv3_bs4096_highmn40_inferred_maxtdelta_5e-3/2/models/running_ckpt.pth"
fn = "/data/t-achetan/dumps/expC_dist_learner_for_adv_ex/rdm_concspheres_test/rdm_concspheres_k50n500_noninfdist_moreoffmfldv3_bs4096_highmn40_inferred_maxtdelta_1e-3/3/models/running_ckpt_epoch_990.pth"

dump = torch.load(fn)
dump["loss"], dump["val_loss"], dump["epoch"]

(1.3231288802188655e-06, 6.248548922400812e-06, 990)

In [3]:
model = MTMLPwithNormalisation(input_size=500, output_size=2, use_tanh=False, use_relu=False, weight_norm=False)
model.load_state_dict(dump["model_state_dict"])

<All keys matched successfully>

In [4]:
optimizer = optim.Adam(model.parameters(), lr=1.5e-5)
optimizer.load_state_dict(dump["optimizer_state_dict"])

In [5]:
print(optimizer.param_groups[0]['lr'])


4.999999999999999e-07


In [6]:
last_lr = dump["scheduler_state_dict"]["_last_lr"][0]
scheduler_params = {"warmup": 10, "cooldown": 700}
num_epochs = 1000
# lr_sched_factor = lambda epoch: (epoch + 991) / (scheduler_params["warmup"]) if (epoch + 991) <= scheduler_params["warmup"] else (1 if (epoch + 991) > scheduler_params["warmup"] and (epoch + 991) < scheduler_params["cooldown"] else max(0, 1 + (1 / (scheduler_params["cooldown"] - (num_epochs + 991))) * ((epoch + 991) - scheduler_params["cooldown"])))
lr_sched_factor = lambda epoch: epoch / (scheduler_params["warmup"]) if epoch <= scheduler_params["warmup"] else (1 if epoch > scheduler_params["warmup"] and epoch < scheduler_params["cooldown"] else max(0, 1 + ((1 / (scheduler_params["cooldown"] - num_epochs)) * (epoch - scheduler_params["cooldown"])) ))
scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_sched_factor, last_epoch=990)
scheduler.load_state_dict(dump["scheduler_state_dict"])

In [7]:
scheduler.get_last_lr()

[4.999999999999999e-07]

In [8]:
for i in range(num_epochs-991):
    print("before step:", scheduler.get_last_lr())
    print(i + 991, 1 / (scheduler_params["cooldown"] - num_epochs), lr_sched_factor(i + 991), lr_sched_factor(i + 991) * 1.5e-05)
    optimizer.step()
    scheduler.step()
    print("after step:", scheduler.get_last_lr())

before step: [4.999999999999999e-07]
991 -0.0033333333333333335 0.029999999999999916 4.4999999999999876e-07
after step: [4.4999999999999876e-07]
before step: [4.4999999999999876e-07]
992 -0.0033333333333333335 0.026666666666666616 3.9999999999999924e-07
after step: [3.9999999999999924e-07]
before step: [3.9999999999999924e-07]
993 -0.0033333333333333335 0.023333333333333317 3.4999999999999977e-07
after step: [3.4999999999999977e-07]
before step: [3.4999999999999977e-07]
994 -0.0033333333333333335 0.019999999999999907 2.999999999999986e-07
after step: [2.999999999999986e-07]
before step: [2.999999999999986e-07]
995 -0.0033333333333333335 0.016666666666666607 2.4999999999999914e-07
after step: [2.4999999999999914e-07]
before step: [2.4999999999999914e-07]
996 -0.0033333333333333335 0.013333333333333308 1.9999999999999962e-07
after step: [1.9999999999999962e-07]
before step: [1.9999999999999962e-07]
997 -0.0033333333333333335 0.009999999999999898 1.4999999999999848e-07
after step: [1.4999

In [9]:
scheduler.get_last_lr()[0]

4.999999999999949e-08

In [10]:
dump["scheduler_state_dict"]

{'base_lrs': [1.5e-05],
 'last_epoch': 990,
 '_step_count': 991,
 'verbose': False,
 '_get_lr_called_within_step': False,
 '_last_lr': [4.999999999999999e-07],
 'lr_lambdas': [None]}

In [11]:
0.029999999999999916 * 4.999999999999999e-07

1.4999999999999952e-08

In [12]:
scheduler.state_dict()

{'base_lrs': [1.5e-05],
 'last_epoch': 999,
 '_step_count': 1000,
 'verbose': False,
 '_get_lr_called_within_step': False,
 '_last_lr': [4.999999999999949e-08],
 'lr_lambdas': [None]}